In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'procedural-environment-generation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4290955%2F7487917%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240322%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240322T083559Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3b8ca034eac44b5335961f4f1568fbd687d4bd14fd8abd38aa0b5e5820e3fab834f834738dafc40d0dd710f3102caaf9ca42f400ac736e3e3541763564796c5427c80b11ae800cd6aee53656a817f15169048f70bd2a84a04128ce8a3a34e8d22db70da69f10d026d08bb3d07b14e062b4bd4d6fa9f9ff23581fe2f44d458e1e41e0386774479cd823c484785b2165d05ef75ea5d60bd7f96d745cf80d2350bea9a5dce67482e49de92aed01e7aabad2b50323310ae66d1226bf627cd8bd61d96204a68ad5621846db7fc21a11dba2ed22517b6f0815353c9c31a7456ce74d2da130f6030c6cfef70585b6b07d078e50c5b359b7acb560c31b7b5ccfca1b1fd3,checkpoint:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4505307%2F7714558%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240322%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240322T083559Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3480de777e12d0ca1bfed7363ab016fd18c35fd988c63b0a9f869b59f35c239b51f8422984e772d604e88733b14bf1e39f2909b508eca869450713c06ac0d861d7f9efeb32601ba48b57fa8d73053f6c97c88741d5b0c52db2119796f5028a038e1862074ea061079361f3378d1270ff60403b291036f256ed7b68ccd46d39b5d94b904841f69428ae29d7fb2232456eabfbd08f6d2300ed10247e22a4547458017d010fdc1159fef603b7bc53c33092e8e60acbb26c4f6e0eefc7b4c2456161df1f2f5f6d1a5e4ee68424fed1eeef9603fdea78571875114bad33a475e6381a28b8310ce6b078617d8669b2e4dc7f02ccd924c11047e9965f085a70b1cbf891'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

from torchvision.utils import save_image
from PIL import Image
import os




In [ ]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # Input is a random noise vector of size nz
            # Output: (ngf * 16) x 4 x 4 (initial layer is doubled for larger image)
            nn.ConvTranspose2d(nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),

            # Output: (ngf * 8) x 8 x 8
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            # Output: (ngf * 4) x 16 x 16
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            # Output: (ngf * 2) x 32 x 32
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            # Output: (ngf) x 64 x 64
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # Output: (nc) x 256 x 256
            nn.ConvTranspose2d(ngf, nc, 4, 4, 0, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 256 x 256
            nn.Conv2d(nc, ndf, 4, 4, 0, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf) x 64 x 64
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf*2) x 32 x 32
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf*4) x 16 x 16
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf*8) x 8 x 8
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
dir_data = "/kaggle/input/procedural-environment-generation/original_raw/original_raw"
img_shape = (256,256,1)
img_paths = [dir_data + '/' + file for file in os.listdir(dir_data)]
img_paths = img_paths[:-6]
img_paths = np.sort(img_paths)

In [ ]:
def load_images(img_paths):
    for image_path in img_paths:
        try:
            img = Image.open(image_path)
            img_array = np.array(img)

            # Check if the difference between max and min values is greater than threshold
            if np.max(img_array) - np.min(img_array) < 50:
                continue  # Skip this image if condition is not met

            # Check for NaN values and replace them with zero
            img_array = np.nan_to_num(img_array)

            # Normalize the image array
            min_val = np.min(img_array)
            max_val = np.max(img_array)
            if min_val != max_val:
                img_array_normalized = (img_array - min_val) / (max_val - min_val)
            else:
                img_array_normalized = img_array

            # Transpose and expand dimensions
            img_array_normalized = np.transpose(np.expand_dims(np.float32(img_array_normalized), axis=2), (2, 0, 1))

            yield img_array_normalized
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")


In [ ]:
def plot_images(imgs, grid_size = 5):

    fig = plt.figure(figsize = (9, 9))
    plt.axis("off")
    columns = rows = grid_size
    plt.title("Training Images")
    images = load_images(imgs)
    for i in range(1, columns*rows +1):

        fig.add_subplot(rows, columns, i)
        plt.axis("off")
        plt.imshow(np.transpose(next(images), (1,2,0)), cmap = 'gray')

    plt.show()

In [ ]:
dev = 'cuda:0' if torch.cuda.is_available() == True else 'cpu'
device = torch.device(dev)
plot_images(img_paths, 5)

In [ ]:
class Terrains(Dataset):
    def __init__(self, img_paths):
        '''np array containing all paths to all images'''
        self.img_paths = img_paths

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = next(load_images([self.img_paths[idx]]))
        return image

In [ ]:
dset = Terrains(img_paths)
batch_size = 32
shuffle = True
dset

In [ ]:
dataloader = DataLoader(dataset = dset, batch_size = batch_size, shuffle = shuffle)


In [ ]:
import torchvision.utils as vutils
real_batch = next(iter(dataloader))
plt.figure(figsize = (8,8))
plt.axis("off")
plt.title("Training_imgs")
plt.imshow(np.transpose(vutils.make_grid(real_batch.to(device)[:64], padding = 2, normalize=True).cpu(),(1,2,0)), cmap='gray')
plt.show()

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:


# Number of workers for dataloader
workers = 2

#

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
BATCH_SIZE = 64
D_lr = 3e-6
G_lr = 3e-4


In [ ]:
criterion = nn.BCELoss()

real_label = 1.
fake_label = 0.
fixed_noise = torch.randn(batch_size, nz, 1,1,device = device)

workers = 2
num_epochs = 200
lr = 0.0002
beta1 = 0.5
netG = Generator(ngpu).to(device)  # Move to GPU
netD = Discriminator(ngpu).to(device)
netG.apply(weights_init)
netD.apply(weights_init)
optimizerD = optim.Adam(netD.parameters(), lr = D_lr, betas = (beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = G_lr, betas = (beta1, 0.999))

In [ ]:
device

In [ ]:
latent_dim=100



In [ ]:
checkpoint_dir="/kaggle/working/saved_checkpoints"
os.makedirs(checkpoint_dir,exist_ok=True)

In [ ]:
img_list = []
G_losses = []
D_losses = []
iters = 0
import os
# Function to save a checkpoint
epoch=95
print("Starting Training Loop...")
checkp_dir='/kaggle/input/checkpoint'

checkpoint_netG = torch.load(os.path.join(checkp_dir, 'netG_checkpoint_epoch_{}.pth'.format(epoch)))
netG.load_state_dict(checkpoint_netG['model_state_dict'])
optimizerG.load_state_dict(checkpoint_netG['optimizer_state_dict'])
start_epoch_G = checkpoint_netG['epoch'] + 1
loss_G = checkpoint_netG['loss']

# Load discriminator checkpoint
checkpoint_netD = torch.load(os.path.join(checkp_dir, 'netD_checkpoint_epoch_{}.pth'.format(epoch)))
netD.load_state_dict(checkpoint_netD['model_state_dict'])
optimizerD.load_state_dict(checkpoint_netD['optimizer_state_dict'])
start_epoch_D = checkpoint_netD['epoch'] + 1
loss_D = checkpoint_netD['loss']
# Function to load a checkpoint
checkpoint_frequency = 5

for epoch in range(start_epoch_D,num_epochs):
    for i, data in enumerate(dataloader, 0):
         ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        ##train with all reals
        netD.zero_grad()
        #setup batch
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,1,5,5), real_label, dtype=torch.float, device = device)

        #fwd pass thru D
        output = netD(real_cpu)
        #calc loss on real
        errD_real = criterion(output, label)
        #calc grads for D
        errD_real.backward()
        D_x = output.mean().item()

        ##train with all fakes
        #gen batch of latent
        noise = torch.randn(b_size, nz,1,1, device = device)
        #generate fake batch by G
#         print(noise.shape)
        fake = netG(noise)
        label.fill_(fake_label)
        #classify all fakes with D
        output = netD(fake.detach())
        #calc D's loss
        errD_fake = criterion(output, label)
        #calc gradients accumulates with prev grads
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        #compute errD as sum of fake and real
        errD = errD_real + errD_fake

        #update D

        optimizerD.step()


        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################

        netG.zero_grad()
        label.fill_(real_label)# fake labels are real for generator cost
        output = netD(fake)
#         print(output)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        if i%50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if (epoch%9==0) and (i%50 == 0):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            fig = plt.figure(figsize=(8,8))
            plt.axis("off")
            ims = [[ plt.imshow(np.transpose(i,(1,2,0)), cmap = 'gray')]for i in img_list]
            plt.show()

            img_list = []

        iters += 1

    if epoch % checkpoint_frequency == 0:
        # Save generator checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': netG.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': errG.item(),
        }, os.path.join(checkpoint_dir, f'netG_checkpoint_epoch_{epoch}.pth'))

        # Save discriminator checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': errD.item(),
        }, os.path.join(checkpoint_dir, f'netD_checkpoint_epoch_{epoch}.pth'))
    torch.cuda.empty_cache()


In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()